# Experiment of star graph from size 2 to 39 on ibmq_manhattan, with measurement error mitigation proposed by Bravyi et al.

External packages by my own:
- bell_ineq.py \
importing the inequality by Baccari et al.

- graph_lib.py \
importing graph library such as making adjacency list of star graph

- graph_state_lib.py \
importing the circuit preparation functions for graph states

## import libraries

### import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image, display
import pickle
import pprint
import time

In [2]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, IBMQ
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
import qiskit.ignis.mitigation as mit
import qiskit.quantum_info as qi
# from qiskit.ignis.mitigation.measurement import complete_meas_cal, tensored_meas_cal, CompleteMeasFitter, TensoredMeasFitter

### import original library

In [3]:
import importlib
import star_bell_ineq
importlib.reload(star_bell_ineq)
from star_bell_ineq import *
import graph_lib
from graph_lib import *
import graph_state_lib
from graph_state_lib import *

### load account

In [4]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='hirashi-jst')
print(provider)

<AccountProvider for IBMQ(hub='ibm-q-utokyo', group='internal', project='hirashi-jst')>


## create star graph information from size 2 to 39
we test the size from 2 to 39

change the number of 27 of "max_size = len(vertex_list[:27]) # largest graph size" if you want to try up to other size

In [5]:
vertex_list =  [19, 
                25, 
                20, 33,
                18, 21, 32,
                34, 17, 12, 31,
                22, 35, 16, 8, 39, 
                11, 30, 23, 36, 15, 7, 45,
                40, 9, 4, 29, 26, 37, 24, 6, 46,
                14, 44, 49, 3, 28, 5, 47] # qubit layout
max_size = len(vertex_list) # largest graph size
Fs = [[0] for n in range(1, max_size + 1)] # subsets of focused vertices of graph size 1 to max_size
adj_lists = [adjacency_list_for_star_graph(n) for n in range(1, max_size + 1)] # adjacency list of graph size 1 to max_size
graphs = [list_to_matrix(adj_list) for adj_list in adj_lists] # adjacency matrix of graph size 1 to max_size
shots_per_circuit = 1 << 13 # shots per one execution
graph_sizes = [len(adj_list) for adj_list in adj_lists]
system_size = len(vertex_list)

In [6]:
print("max_size:", max_size)
print("system_size:", system_size)
print("shots_per_circuit:", shots_per_circuit)
print("graph_sizes:", graph_sizes)

max_size: 39
system_size: 39
shots_per_circuit: 8192
graph_sizes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [7]:
# indicate how to connect qubits
order = [[0,1],
         [0,2],[1,3],
         [0,4],[2,5],[3,6],
         [3,7],[4,8],[5,9],[6,10],
         [5,11],[7,12],[8,13],[9,14],[10,15],
         [8,16],[10,17],[11,18],[12,19],[13,20],[14,21],[15,22],
         [12,23],[14,24],[16,25],[17,26],[18,27],[19,28],[20,29],[21,30],[22,31],
         [20,32],[22,33],[23,34],[25,35],[26,36],[30,37],[31,38]]

In [8]:
qubit_depths = [0, 
                1, 
                2, 2,
                3, 3, 3, 
                4, 4, 4, 4, 
                5, 5, 5, 5, 5, 
                6, 6, 6, 6, 6, 6, 6, 
                7, 7, 7, 7, 7, 7, 7, 7, 7,
                8, 8, 8, 8, 8, 8, 8]
print(qubit_depths)
print(len(qubit_depths))

[0, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8]
39


## prepare quantum circuits (from size 2 to 39)

In [9]:
qcs = [refocusing_last_star_graph_state(size, order[:size-1], qubit_depths[:size], system_size) for size in range(1, max_size+1)]
qcs_list, nums_divide_list, nums_meas_cal_list, metadatas_list, _, initial_layouts_list = \
    prepare_grouping_star_graph_qcs_list(qcs, 
                                        adj_lists, 
                                        measure_last=True,
                                        optimization_level=0,
                                        initial_layouts=[vertex_list[:system_size]] * len(qcs),
                                        method="CTMP",
                                        mitigation_limit=15)
print("the number of graph state circuits is: ", len(qcs_list))

the number of graph state circuits is:  223


In [10]:
ops_qcs_list = [qc.count_ops() for qc in qcs_list]
print(len(qcs_list), "quantum circuits")
print("length of initial_layouts_list: ", len(initial_layouts_list))

223 quantum circuits
length of initial_layouts_list:  223


In [11]:
qcs_depths = [qc.depth() for qc in qcs[1:]]
print("qcs_depth:", qcs_depths)

qcs_depth: [4, 5, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11]


## cast jobs to ibmq_manhattan and save job ids

In [12]:
device = provider.backends.ibmq_manhattan
backend = provider.get_backend('ibmq_manhattan')

/Users/yangbo/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The `backends` provider attribute is deprecated. Please use `provider.backend` (singular) instead. You can continue to use `provider.backends()` to retrieve all backends.
  """Entry point for launching an IPython kernel.


In [13]:
"""
jobs = execute_circuits(qcs_list,
                        backend='ibmq_manhattan', 
                        provider=provider, 
                        shots=shots_per_circuit,
                        max_experiments=900,
                        optimization_level=1,
                        initial_layout=initial_layouts_list)
print(len(jobs), "jobs")
"""

'\njobs = execute_circuits(qcs_list,\n                        backend=\'ibmq_manhattan\', \n                        provider=provider, \n                        shots=shots_per_circuit,\n                        max_experiments=900,\n                        optimization_level=1,\n                        initial_layout=initial_layouts_list)\nprint(len(jobs), "jobs")\n'

In [14]:
job_ids_filename = "job_ids_manhattan_star.pkl"

In [15]:
# with open(job_ids_filename, "wb") as f:
#     pickle.dump([job.job_id() for job in jobs], f)

In [16]:
# with open("manhattan_properties_star.pkl", "wb") as f:
#     pickle.dump(device.properties(), f)

## retrieve jobs

In [17]:
with open(job_ids_filename, "rb") as f:
     job_ids = pickle.load(f)

In [18]:
# using job_ids_to_result function
results_jobs_list = job_ids_to_result(job_ids, device)
print(len(results_jobs_list))

1


## measurement error mitigation (CTMP)

In [19]:
results_list = flatten_results_jobs_list(results_jobs_list)
print("length of results_list: ", len(results_list))
del results_jobs_list

length of results_list:  223


In [20]:
def arrange_results_list_for_grouping_star_graph(results_list, nums_divide_list, nums_meas_cal_list, limit=100):
    pos = 0
    results_graph_states = []
    results_meas_cal = []
    for i, (nums_divide, nums_meas_cal) in enumerate(zip(nums_divide_list, nums_meas_cal_list)): # graph wise
        for num_divide in nums_divide: # term wise
            results_graph_states += results_list[pos:pos + num_divide]
            pos += num_divide
        if i < limit: # start from 2
            for num_meas_cal in nums_meas_cal: # term wise
                results_meas_cal.append(merge_results(results_list[pos:pos + num_meas_cal]))
                pos += num_meas_cal
    return results_graph_states, results_meas_cal

In [21]:
results_graph_states, results_meas_cal = arrange_results_list_for_grouping_star_graph(results_list, nums_divide_list, nums_meas_cal_list, limit=15)
print("length of results_graph_states: ", len(results_graph_states))
print("length of results_meas_cal: ", len(results_meas_cal))
print("length of results_metadata_list: ", len(metadatas_list))
del results_list

length of results_graph_states:  76
length of results_meas_cal:  14
length of results_metadata_list:  38


In [22]:
# meas_mitigator_list = prepare_meas_mitigator_list(results_meas_cal, metadatas_list[:14])
# print("length of meas_mitigator_list: ", len(meas_mitigator_list))

In [23]:
counts_dict_list = results_list_to_counts_dict_list(results_graph_states)
print("length of counts_dict_list: ", len(counts_dict_list))

length of counts_dict_list:  76


### Tensored

In [24]:
def arrange_results_list_tensored(results_list, nums_divide_list, nums_meas_cal_list, limit=100):
    pos = 0
    results_graph_states = []
    results_meas_cal = []
    for i, (nums_divide, nums_meas_cal) in enumerate(zip(nums_divide_list, nums_meas_cal_list)): # graph wise
        for num_divide in nums_divide: # term wise
            results_graph_states += results_list[pos:pos + num_divide]
            pos += num_divide
        if i < limit: # start from 2
            for num_meas_cal in nums_meas_cal: # term wise
                results_meas_cal.append(merge_results(results_list[pos:pos + 2]))
                pos += num_meas_cal
    return results_graph_states, results_meas_cal

In [25]:
with open(job_ids_filename, "rb") as f:
     job_ids = pickle.load(f)
results_jobs_list = job_ids_to_result(job_ids, device)
print(len(results_jobs_list))
results_list = flatten_results_jobs_list(results_jobs_list)
print("length of results_list: ", len(results_list))
del results_jobs_list
_, results_meas_cal = arrange_results_list_tensored(results_list, nums_divide_list, nums_meas_cal_list, limit=15)
print("length of results_meas_cal: ", len(results_meas_cal))
del results_list

1
length of results_list:  223
length of results_meas_cal:  14


In [26]:
tensored_meas_mitigator_list = []
pos = 0
for n in range(2,16):
    qr = qiskit.QuantumRegister(n)
    mit_pattern = [[i] for i in range(n)]
    _, state_labels = tensored_meas_cal(mit_pattern = mit_pattern, qr=qr, circlabel='mcal')
    tensored_meas_mitigator_list.append( TensoredMeasFitter(results_meas_cal[pos:pos+2], mit_pattern=mit_pattern).filter )
    pos += 2
del results_meas_cal

## compute correlation

In [27]:
# without CTMP mitigation
corr_list_real, stddev_list_real, Es_list_real, Ds_list_real = analyze_circuits_for_star_graph(adj_lists, 
                                                                                            counts_dict_list,
                                                                                            None)

graph size: 1
skipped

graph size: 2
total correlation: 2.611599459421409 

graph size: 3
total correlation: 5.006716520501039 

graph size: 4
total correlation: 6.98889426991118 

graph size: 5
total correlation: 7.239903366553084 

graph size: 6
total correlation: 9.498985236330222 

graph size: 7
total correlation: 11.859920866092729 

graph size: 8
total correlation: 9.1872091506801 

graph size: 9
total correlation: 10.365260096680345 

graph size: 10
total correlation: 7.209865139031976 

graph size: 11
total correlation: 10.136002819967517 

graph size: 12
total correlation: 16.183008760251305 

graph size: 13
total correlation: 16.9591689380381 

graph size: 14
total correlation: -4.371080004795748 

graph size: 15
total correlation: 14.152148366237988 

graph size: 16
total correlation: 16.552099165079635 

graph size: 17
total correlation: 14.725636825003054 

graph size: 18
total correlation: 17.460841864338907 

graph size: 19
total correlation: 16.256205360647566 

graph s

In [28]:
def analyze_circuits_tensored(adj_lists, counts_dict_list, tensored_meas_mitigator_list=None, limit=100, method='fast', threshold=0.1):
    """
    Input
        adj_lists         : list of adjacency list
        counts_list       : list of int list (list of counts)
        meas_mitigator : measurement mitigator
    Output
        expval_all_list : list of float (correlation of each graph)
        stddev_all_list : list of float (standard deviation of each graph)
        Es_all_list   : list of list (term-wise correlation of each graph)
        Ds_all_list   : list of list (term-wise stddev of each graph)
    """
    expval_all_list, stddev_all_list, Es_all_list, Ds_all_list = [], [], [], []
    # begin = 0
    for adj_list in adj_lists:
        t1 = time.time()
        n = len(adj_list)
        if n > limit:
            break
        print("graph size:", n)
        if n <= 1:
            print("skipped\n")
            expval_all_list.append(0)
            stddev_all_list.append(0)
            Es_all_list.append([])
            Ds_all_list.append([])
            continue

        # for the first term
        counts = tensored_meas_mitigator_list[n - 2].apply( counts_dict_list[2 * (n - 2)], method=method, threshold=threshold ) # n = 2 -> 0, n = 10 -> 16
        expval1, stddev1 = mit.expectation_value(counts, 
                                                 qubits=range(n),
                                                 clbits=range(n),
                                                 meas_mitigator=None)
        Es_1, Ds_1 = [expval1], [stddev1]
        # begin += 1 # update index of counts_dict_list


        # for the second term
        Es_2, Ds_2 = [], []
        sum_expval2, sum_stddev2 = 0, 0
        for pos in range(1, n): # recover the two qubit expectation values
            counts = tensored_meas_mitigator_list[n - 2].apply( counts_dict_list[2 * (n - 2) + 1], method=method, threshold=threshold ) # n = 2 -> 1, n = 10 -> 17
            expval2, stddev2 = mit.expectation_value(extract_two_qubit_counts(counts, 0, pos), 
                                                     qubits=[0,pos],
                                                     clbits=[0,pos],
                                                     meas_mitigator=None)
            Es_2.append(expval2)
            Ds_2.append(stddev2)
        sum_stddev2 = compute_stddev_of_grouping(Ds_2)
        # begin += 1 # update index of counts_dict_list

        sum_expval = np.sqrt(2) * ((n - 1) * sum(Es_1) + sum(Es_2))
        sum_stddev = np.sqrt(2 * ((stddev1 * (n - 1)) ** 2 +  sum_stddev2 ** 2) )
        Es = [Es_1, Es_2]
        Ds = [Ds_1, Ds_2]

        expval_all_list.append(sum_expval)
        stddev_all_list.append(sum_stddev)
        Es_all_list.append(Es)
        Ds_all_list.append(Ds)
        t2 = time.time()
        print("time:", t2 - t1)
        print("total correlation:", sum_expval, "\n")
    return expval_all_list, stddev_all_list, Es_all_list, Ds_all_list

In [29]:
# with tensored mitigation
tensored_corr_list_real, tensored_stddev_list_real, tensored_Es_list_real, tensored_Ds_list_real = analyze_circuits_tensored(adj_lists, 
                                                                                                                            counts_dict_list,
                                                                                                                            tensored_meas_mitigator_list,
                                                                                                                            limit=11,
                                                                                                                            method='fast',
                                                                                                                            threshold=0.1)

graph size: 1
skipped

graph size: 2
{'00': 4204, '01': 149, '10': 135, '11': 3704}
{'00': 4236.63472940583, '11': 3794.650750599218}
{'00': 4271.750145385088, '11': 3902.211872792729}

[0.35093485 0.64906515] 2
((0, 8192), (0, 8192)) 2
[4271.75014539 3902.21187279] 2
{'00': 4271.750145385088, '11': 3902.211872792729} 2
{'00': 4118, '01': 203, '10': 141, '11': 3730}
{'00': 4148.552983474203, '11': 3821.2434086035073}
{'00': 4182.938332467252, '11': 3929.5583119286803}

[0.71451719 0.28548281] 2
((0, 8192), (0, 8192)) 2
[4182.93833247 3929.55831193] 2
{'00': 4182.938332467252, '11': 3929.5583119286803} 2
time: 0.008838891983032227
total correlation: 2.8284271247461903 

graph size: 3
{'000': 2137, '001': 129, '010': 166, '011': 1733, '100': 169, '101': 1945, '110': 1814, '111': 99}
{'000': 2151.947948831055, '011': 1773.2059991177766, '101': 1990.270590459386, '110': 1826.940765013549}
{'000': 2176.156671577612, '011': 1847.6405271085143, '101': 2012.6604948904994, '110': 1903.630937265

ValueError: unexpected array size: new_size=1, got array with arr_size=0


In [ ]:
# theoretical bounds
c_bounds = [2 * i for i in range(max_size)]
q_bounds = [2 * np.sqrt(2) * i for i in range(max_size)]

## Plot correlations

In [ ]:
plt.clf()
plt.plot(list(range(1, max_size + 1)), c_bounds, label="Classical bounds", marker="o", markersize=3)
plt.plot(list(range(1, max_size + 1)), q_bounds, label="Quantum bounds", marker="x", markersize=3)
plt.errorbar(list(range(1, max_size + 1)), corr_list_real, label="Actual correlations", fmt="v", yerr=stddev_list_real, capsize=3)
plt.errorbar(list(range(1, 11 + 1)), tensored_corr_list_real, label="Mitigated correlations", fmt="*", yerr=tensored_stddev_list_real, capsize=3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel("graph size")
plt.ylabel("correlation")
plt.xticks(list(range(max_size + 2)))
plt.title("correlation of star graphs (ibmq_manhattan)")
plt.show()

In [ ]:
max_size = 39
plt.clf()
plt.plot(list(range(1, max_size + 1)), c_bounds, label="Classical bounds") # , marker="o", markersize=3)
plt.plot(list(range(1, max_size + 1)), q_bounds, label="Quantum bounds") # , marker="x", markersize=3)
plt.errorbar(list(range(1, max_size + 1)), corr_list_real, label="Actual correlations", fmt="o", yerr=stddev_list_real, capsize=3)
plt.errorbar(list(range(1, 11 + 1)), tensored_corr_list_real, label="Mitigated correlations", fmt="*", yerr=tensored_stddev_list_real, capsize=3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel("graph size")
plt.ylabel("correlation")
plt.xticks(list(range(max_size + 2))[::2])
plt.title("correlation of star graphs (ibmq_sydney)")
plt.show()

In [ ]:
each_corr_list = [[0]] + [E_1 + Es_2 for E_1, Es_2 in Es_list_real[1:]]
tp_each_corr_list = []
for i in range(len(each_corr_list)):
    tp_each_corr_list.append([each_corr_list[j][i] for j in range(i, len(each_corr_list))])
# pprint.pprint(tp_each_corr_list)

In [ ]:
each_stddev_list = [[0]] + [D_1 + Ds_2 for D_1, Ds_2 in Ds_list_real[1:]]
tp_each_stddev_list = []
for i in range(len(each_stddev_list)):
    tp_each_stddev_list.append([each_stddev_list[j][i] for j in range(i, len(each_stddev_list))])
# pprint.pprint(tp_each_corr_list)

In [ ]:
max_size = 39
plt.clf()
for i, (each_corr, each_stddev) in enumerate(zip(tp_each_corr_list, tp_each_stddev_list)):
    if i == 0:
        plt.plot(list(range(2, max_size + 1)), each_corr[1:], color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))])
        plt.errorbar(list(range(2, max_size + 1)), each_corr[1:], label=str(i), color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))], fmt="x", yerr=each_stddev[1:], capsize=3)
    else:
        plt.plot(list(range(i+1, max_size + 1)), each_corr, color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))])
        plt.errorbar(list(range(i+1, max_size + 1)), each_corr, label=str(i), color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))], fmt="v", yerr=each_stddev, capsize=3)
#  plt.plot(list(range(1, max_size + 1)), damped_vibration, label="damped vibration", color=[0, 0, 1])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel("graph size")
plt.ylabel("correlation")
plt.xticks(list(range(max_size + 2)))
plt.ylim([-1,1])
plt.title("correlation of each term (ibmq_manhattan)")
plt.show()

In [ ]:
mitigated_each_corr_list = [[0]] + [E_1 + Es_2 for E_1, Es_2 in tensored_Es_list_real[1:]]
mitigated_tp_each_corr_list = []
for i in range(len(mitigated_each_corr_list)):
    mitigated_tp_each_corr_list.append([mitigated_each_corr_list[j][i] for j in range(i, len(mitigated_each_corr_list))])
# pprint.pprint(tp_each_corr_list)

In [ ]:
mitigated_each_stddev_list = [[0]] + [D_1 + Ds_2 for D_1, Ds_2 in tensored_Ds_list_real[1:]]
mitigated_tp_each_stddev_list = []
for i in range(len(mitigated_each_stddev_list)):
    mitigated_tp_each_stddev_list.append([mitigated_each_stddev_list[j][i] for j in range(i, len(mitigated_each_stddev_list))])
# pprint.pprint(tp_each_corr_list)

In [ ]:
max_size = 11
plt.clf()
for i, (each_corr, each_stddev) in enumerate(zip(mitigated_tp_each_corr_list, mitigated_tp_each_stddev_list)):
    if i == 0:
        plt.plot(list(range(2, max_size + 1)), each_corr[1:], color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))])
        plt.errorbar(list(range(2, max_size + 1)), each_corr[1:], label=str(i), color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))], fmt="x", yerr=each_stddev[1:], capsize=3)
    else:
        plt.plot(list(range(i+1, max_size + 1)), each_corr, color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))])
        plt.errorbar(list(range(i+1, max_size + 1)), each_corr, label=str(i), color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))], fmt="v", yerr=each_stddev, capsize=3)
#  plt.plot(list(range(1, max_size + 1)), damped_vibration, label="damped vibration", color=[0, 0, 1])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel("graph size")
plt.ylabel("correlation")
plt.xticks(list(range(max_size + 2)))
plt.ylim([-1,1])
plt.title("mitigated correlation of each term (ibmq_manhattan)")
plt.show()

In [ ]:
max_size = 11
plt.clf()
for i, (each_corr, each_stddev) in enumerate(zip(mitigated_tp_each_corr_list, mitigated_tp_each_stddev_list)):
    if i == 0:
        plt.plot(list(range(2, max_size + 1)), each_corr[1:], color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))])
        plt.errorbar(list(range(2, max_size + 1)), each_corr[1:], label=str(i), color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))], fmt="x", yerr=each_stddev[1:], capsize=3)
    else:
        plt.plot(list(range(i+1, max_size + 1)), each_corr, color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))])
        plt.errorbar(list(range(i+1, max_size + 1)), each_corr, label=str(i), color=[min(1, 2*i/max_size), 0, min(1, 2*(1-i/max_size))], fmt="v", yerr=each_stddev, capsize=3)
#  plt.plot(list(range(1, max_size + 1)), damped_vibration, label="damped vibration", color=[0, 0, 1])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel("graph size")
plt.ylabel("correlation")
plt.xticks(list(range(max_size + 2)))
plt.ylim([-1,1.1])
plt.title("mitigated correlation of each term (ibmq_manhattan)")
plt.show()